In [1]:
import torch
from torch import nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from preprocess import Data
from model_v1 import ResNet18
from model_v1 import ResBlock
from torch.utils.data import TensorDataset, DataLoader

In [2]:
dataobj = Data()
batch_size = 32
epochs = 25
X_train, y_train, X_test, y_test, X_val, y_val = dataobj.preprocess()

In [3]:
X_train.shape

(4325, 150, 150, 1)

In [4]:
tensor_X_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)
tensor_X_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)
tensor_X_val = torch.Tensor(X_val)
tensor_y_val = torch.Tensor(y_val)

In [5]:
print(tensor_X_test.shape)
print(tensor_y_test.size(0))

torch.Size([624, 150, 150, 1])
624


In [6]:
train_set = TensorDataset(tensor_X_train, tensor_y_train)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = TensorDataset(tensor_X_test, tensor_y_test)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [7]:
model = ResNet18(150, ResBlock, 1)

In [8]:
from torch.autograd import Variable

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    model.train()
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            outputs = outputs.reshape(-1)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()
            

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 20 == 19:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.8f' %
                      (epoch + 1, i + 1, running_loss / 32))
                # zero the loss
                running_loss = 0.0
                print("Outputs: ", outputs)
                print("Outputs.data: ", outputs.data)
                #_, predicted = torch.max(outputs.data, 1)
                #print("Predicted: ", predicted)
                #print("_ :", _)

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [9]:

# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 3, 0)))
    plt.show()


# Function to test the model with a batch of images and show the labels predictions
def testBatch():
    # get batch of images from the test DataLoader  
    images, labels = next(iter(test_loader))

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))
   
    # Show the real labels on the screen 
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
                               for j in range(batch_size)))
  
    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)
    
    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)
    
    # Let's show the predicted labels on the screen to compare with the real ones
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
                              for j in range(batch_size)))

In [10]:
train(5)

The model will be running on cpu device
[1,    20] loss: 0.24924869
Outputs:  tensor([0.9937, 0.7075, 0.5330, 0.9945, 0.2842, 0.9897, 0.0068, 0.9994, 0.8545,
        0.9992, 0.9974, 0.0051, 0.9116, 0.9983, 0.9992, 0.9975, 0.0306, 0.9706,
        0.5135, 0.9965, 0.0101, 0.9997, 0.9989, 0.9961, 0.9987, 0.9930, 0.0817,
        0.2661, 0.8222, 0.8539, 0.9986, 0.9994],
       grad_fn=<ReshapeAliasBackward0>)
Outputs.data:  tensor([0.9937, 0.7075, 0.5330, 0.9945, 0.2842, 0.9897, 0.0068, 0.9994, 0.8545,
        0.9992, 0.9974, 0.0051, 0.9116, 0.9983, 0.9992, 0.9975, 0.0306, 0.9706,
        0.5135, 0.9965, 0.0101, 0.9997, 0.9989, 0.9961, 0.9987, 0.9930, 0.0817,
        0.2661, 0.8222, 0.8539, 0.9986, 0.9994])
[1,    40] loss: 0.14902582
Outputs:  tensor([0.9226, 0.9976, 0.9402, 0.9761, 0.9979, 0.9791, 0.9910, 0.0510, 0.9820,
        0.0052, 0.2515, 0.0046, 0.9978, 0.9998, 0.7048, 0.8603, 0.1223, 0.3176,
        0.1664, 0.2508, 0.9997, 0.9881, 0.9959, 0.1092, 0.9330, 0.9998, 0.9830,
        0.9

For epoch 2 the test accuracy over the whole test set is 37 %
[3,    20] loss: 0.18707536
Outputs:  tensor([0.9497, 0.2071, 0.0554, 0.5326, 0.9686, 0.9481, 0.9302, 0.1897, 0.9757,
        0.9781, 0.9616, 0.5658, 0.9741, 0.9742, 0.7537, 0.9785, 0.7081, 0.9279,
        0.9790, 0.9406, 0.1610, 0.9387, 0.3445, 0.9351, 0.0856, 0.9559, 0.9549,
        0.8046, 0.5761, 0.9618, 0.9473, 0.9675],
       grad_fn=<ReshapeAliasBackward0>)
Outputs.data:  tensor([0.9497, 0.2071, 0.0554, 0.5326, 0.9686, 0.9481, 0.9302, 0.1897, 0.9757,
        0.9781, 0.9616, 0.5658, 0.9741, 0.9742, 0.7537, 0.9785, 0.7081, 0.9279,
        0.9790, 0.9406, 0.1610, 0.9387, 0.3445, 0.9351, 0.0856, 0.9559, 0.9549,
        0.8046, 0.5761, 0.9618, 0.9473, 0.9675])
[3,    40] loss: 0.12792069
Outputs:  tensor([0.9992, 0.9997, 0.0140, 0.4744, 0.8537, 0.0129, 0.5986, 0.0074, 0.9937,
        0.0468, 0.9994, 0.9967, 0.9995, 0.9997, 0.5233, 0.9981, 0.9956, 0.9937,
        0.0476, 0.9909, 0.9688, 0.9972, 0.9799, 0.9953, 0.9972, 0.709

[5,    20] loss: 0.12771556
Outputs:  tensor([0.8369, 0.9955, 0.9186, 0.9906, 0.0974, 0.9991, 0.1987, 0.8529, 0.9864,
        0.9868, 0.9988, 0.9989, 0.9684, 0.9989, 0.9990, 0.9979, 0.0828, 0.6914,
        0.0302, 0.1939, 0.6279, 0.0385, 0.9938, 0.9982, 0.9997, 0.9864, 0.9998,
        0.9844, 0.9974, 0.1114, 0.9980, 0.9567],
       grad_fn=<ReshapeAliasBackward0>)
Outputs.data:  tensor([0.8369, 0.9955, 0.9186, 0.9906, 0.0974, 0.9991, 0.1987, 0.8529, 0.9864,
        0.9868, 0.9988, 0.9989, 0.9684, 0.9989, 0.9990, 0.9979, 0.0828, 0.6914,
        0.0302, 0.1939, 0.6279, 0.0385, 0.9938, 0.9982, 0.9997, 0.9864, 0.9998,
        0.9844, 0.9974, 0.1114, 0.9980, 0.9567])
[5,    40] loss: 0.13697059
Outputs:  tensor([0.2573, 0.9994, 0.9967, 0.9899, 0.9973, 0.9979, 0.9031, 0.9991, 0.9910,
        0.9957, 0.9863, 0.9981, 0.9958, 0.9925, 0.3823, 0.9871, 0.9910, 0.9944,
        0.9884, 0.7932, 0.9948, 0.9747, 0.9984, 0.9937, 0.9959, 0.9967, 0.2014,
        0.6986, 0.9995, 0.9981, 0.9867, 0.9919],
  

In [11]:
print(X_train)

[[[[0.43529412]
   [0.45490196]
   [0.48235294]
   ...
   [0.2745098 ]
   [0.25098039]
   [0.22352941]]

  [[0.22352941]
   [0.24705882]
   [0.28235294]
   ...
   [0.26666667]
   [0.24313725]
   [0.20784314]]

  [[0.15294118]
   [0.18431373]
   [0.21568627]
   ...
   [0.2627451 ]
   [0.23921569]
   [0.2       ]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]]


 [[[0.24313725]
   [0.21960784]
   [0.21176471]
   ...
   [0.8       ]
   [0.81176471]
   [0.80392157]]

  [[0.24313725]
   [0.23137255]
   [0.22745098]
   ...
   [0.8       ]
   [0.8       ]
   [0.8       ]]

  [[0.25098039]
   [0.24313725]
   [0.22745098]
   ...
   [0.8       ]
   [0.80392157]
   [0.81568627]]

  ...

  [[0.05882353]
   [0.05098039]
   [0.0

In [12]:
print(y_train)

[0 1 1 ... 1 1 1]
